In [1]:
from pyzotero import zotero as pyzt
import pandas as pd
import sqlite3
import yaml
import re

In [3]:
# Get your zotero configuration
with open('../../data/zotero.yaml', 'r') as file:
    cf = yaml.safe_load(file)

In [6]:
# Access SAEF group library
zot = pyzt.Zotero(library_id = cf['library']['id'], library_type=cf['library']['type'], api_key=cf['library']['key'])

In [85]:
# Import Filemaker Person keys 
pids = pd.read_csv('../../data/Peoplekey.csv', usecols=[0, 1]) 
pids.columns = ['NameKey', 'ID_Person']

In [23]:
# Retrieve all top-level items
saef_library = zot.everything(zot.top())

In [126]:
# https://pyzotero.readthedocs.io/en/latest/#zotero.Zotero.collections
def matched_library(library):
    items = library
    result = pd.DataFrame(columns=['key', 'itemType', 'title', 'rights', 'pubyr', 'name', 'id_person'])
                          
    for x in items:
        item  = x['data']
        pubyr = re.findall( '\d{4}', item['date'] )[0]  # Match 4 consequtive numbers in a string i.e. YYYY
        extra = item['extra']
        saef_author_list = re.findall('saef:.*', extra) # Use saef keyword as an identifier
        
        for names in saef_author_list:
            names_tidy  = re.sub('saef:', '', names)
            names_split = names_tidy.strip().split(';') # Create a list of saef names
            
            for name in names_split:
                # Find FileMaker person key
                id_person = pids[pids.NameKey == name]['ID_Person'].to_string()
                # 'key', ['Item Type', 'Title', 'Journal', 'publication', 'rights', 'date', author, id]
                store = [item['key'], item['itemType'], item['title'], item['rights'], pubyr, name, id_person]
                result.loc[len(result)] = store
    return result

In [127]:
# Get a Filemaker people_Outputs ready object
bucket = matched_library(saef_library)

In [128]:
bucket

,key,itemType,title,rights,pubyr,name,id_person
0,KWBU9DB4,videoRecording,Antarctic Futures Seminar 2 | Antarctic Film N...,oa,2022,MelindaWaterman,113 DEB79D29-B3D0-AD40-9775-D35D57A63777
1,KWBU9DB4,videoRecording,Antarctic Futures Seminar 2 | Antarctic Film N...,oa,2022,GeorgiaWatson,45 E485E5D8-D06C-C646-98C8-758AD0BBCDA9
2,KWBU9DB4,videoRecording,Antarctic Futures Seminar 2 | Antarctic Film N...,oa,2022,KrystalRandall,86 A9CC637C-8FB9-514F-AEDE-3528CDCEEA5C
3,W554N7DH,webpage,Biodiversity unbalanced as ice-free Antarctic ...,Copyright Queensland University of Technology ...,2022,JustineShaw,77 FBC44D1A-F9B8-D744-9B50-F4CD71E237D1
4,W554N7DH,webpage,Biodiversity unbalanced as ice-free Antarctic ...,Copyright Queensland University of Technology ...,2022,JasmineLee,61 E6069ABD-ACFB-A84A-9848-FF144AFE175A
...,...,...,...,...,...,...,...
65,9B6LFADN,radioBroadcast,Antarctic ecosystems under double pressure,oa,2023,AleksTerauds,"Series([], )"
66,C49RR9A9,radioBroadcast,Interview with Kate Helmstedt,oa,2022,KateHelmstedt,80 81E17FF3-F45C-074F-9A24-83BC9189D10E
67,QD9D7I8E,radioBroadcast,,oa,2022,EmmaJohnston,39 54567B97-2A61-8C47-8046-9648207C2437
68,TBBBATKP,radioBroadcast,Native species thrive on Curtis Island after r...,oa,2022,JustineShaw,77 FBC44D1A-F9B8-D744-9B50-F4CD71E237D1
